# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import re

from ccd_functions import read_distict, cal_counts, cal_district_numbers

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = read_distict()
dist.head()

,council,tract,percent
0,1.0,192000,0.221309
1,1.0,191900,0.000049
2,1.0,192300,0.185373
3,1.0,192100,0.572719
4,1.0,191304,0.786238


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP02-Data.csv'
filename2 = 'ACSDP5Y2021.DP02-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True).replace('N',np.nan)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp02_0001e,dp02_0001ea,dp02_0001m,dp02_0001ma,dp02_0002e,dp02_0002m,dp02_0002ma,dp02_0002ea,dp02_0003e,...,dp02_0152pma,dp02_0153pe,dp02_0153pm,dp02_0153pma,dp02_0153pea,dp02_0154pe,dp02_0154pm,dp02_0154pma,dp02_0154pea,unnamed: 1234
0,110100,1895,NaN,286,NaN,283,155,NaN,NaN,15,...,NaN,92.3,5.3,NaN,NaN,89.0,6.3,NaN,NaN,NaN
1,110300,1393,NaN,251,NaN,335,168,NaN,NaN,241,...,NaN,94.9,5.0,NaN,NaN,75.6,12.3,NaN,NaN,NaN
2,110500,921,NaN,133,NaN,128,51,NaN,NaN,25,...,NaN,79.9,5.7,NaN,NaN,73.5,7.3,NaN,NaN,NaN
3,110600,1551,NaN,269,NaN,126,77,NaN,NaN,7,...,NaN,69.3,10.1,NaN,NaN,57.8,9.9,NaN,NaN,NaN
4,110700,513,NaN,104,NaN,134,56,NaN,NaN,84,...,NaN,86.7,5.9,NaN,NaN,64.1,14.6,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['HOUSEHOLDS BY TYPE', 'RELATIONSHIP', 'MARITAL STATUS',
       'FERTILITY', 'GRANDPARENTS', 'SCHOOL ENROLLMENT',
       'EDUCATIONAL ATTAINMENT', 'VETERAN STATUS',
       'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION',
       'RESIDENCE 1 YEAR AGO', 'PLACE OF BIRTH',
       'U.S. CITIZENSHIP STATUS', 'YEAR OF ENTRY',
       'WORLD REGION OF BIRTH OF FOREIGN BORN', 'LANGUAGE SPOKEN AT HOME',
       'ANCESTRY', 'COMPUTERS AND INTERNET USE'], dtype=object)

## Now combine and calculate! 

### Labor force

In [6]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
print()
    
final1 = cal_district_numbers(dist, dataall, labels, labels_check, 'dp02_0001e')
final1

HOUSEHOLDS BY TYPE

Estimate!!HOUSEHOLDS BY TYPE!!Total households
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!F

,0,1,2,3,4,5,6,7,8,9,10
Total households counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Total households counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Total households moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Married-couple household counts,90560.02,19425.14,16828.77,19260.36,18590.37,15479.05,19998.06,23408.47,21223.13,24043.96,25152.67
...,...,...,...,...,...,...,...,...,...,...,...
Average household size moa percent,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
Average family size counts,246.93,106.32,101.12,108.22,88.62,124.20,76.22,99.24,79.19,72.42,82.45
Average family size counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Average family size moe,2.56,2.23,2.80,2.01,1.85,2.52,1.95,1.89,1.47,1.28,1.47


### Relationship

In [7]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

final2 = cal_district_numbers(dist, dataall, labels, labels_check, 'dp02_0018e')    
final2

RELATIONSHIP

Estimate!!RELATIONSHIP!!Population in households
Estimate!!RELATIONSHIP!!Population in households!!Householder
Estimate!!RELATIONSHIP!!Population in households!!Spouse
Estimate!!RELATIONSHIP!!Population in households!!Unmarried partner
Estimate!!RELATIONSHIP!!Population in households!!Child
Estimate!!RELATIONSHIP!!Population in households!!Other relatives
Estimate!!RELATIONSHIP!!Population in households!!Other nonrelatives


,0,1,2,3,4,5,6,7,8,9,10
Population in households counts,474555.20,133671.43,127973.35,139468.67,131885.51,137244.43,129553.04,150190.64,141326.05,119990.44,140447.26
Population in households counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.08
Population in households moe,8124.80,3877.69,4231.68,4022.83,4149.41,4403.04,4394.63,4542.80,3763.91,3337.18,3832.19
Population in households moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Householder counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Householder counts percent,0.09,0.03,0.03,0.03,0.02,0.03,0.02,0.03,0.03,0.03,0.03
Householder moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Householder moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Spouse counts,90004.05,19520.34,16916.24,19765.73,18364.27,15748.62,19849.79,23225.05,21148.03,23912.68,24919.19
Spouse counts percent,0.05,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


### Marital Status

In [8]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final3 = cal_district_numbers(dist, dataall, labels, labels_check, 'dp02_0025e')
final3

MARITAL STATUS

Estimate!!MARITAL STATUS!!Males 15 years and over
Estimate!!MARITAL STATUS!!Males 15 years and over!!Never married
Estimate!!MARITAL STATUS!!Males 15 years and over!!Now married, except separated
Estimate!!MARITAL STATUS!!Males 15 years and over!!Separated
Estimate!!MARITAL STATUS!!Males 15 years and over!!Widowed
Estimate!!MARITAL STATUS!!Males 15 years and over!!Divorced
Estimate!!MARITAL STATUS!!Females 15 years and over
Estimate!!MARITAL STATUS!!Females 15 years and over!!Never married
Estimate!!MARITAL STATUS!!Females 15 years and over!!Now married, except separated
Estimate!!MARITAL STATUS!!Females 15 years and over!!Separated
Estimate!!MARITAL STATUS!!Females 15 years and over!!Widowed
Estimate!!MARITAL STATUS!!Females 15 years and over!!Divorced


,0,1,2,3,4,5,6,7,8,9,10
Males 15 years and over counts,184936.35,55515.43,48974.17,54653.95,48904.63,54479.47,49056.83,59446.96,61196.37,47129.61,54416.22
Males 15 years and over counts percent,0.26,0.08,0.07,0.08,0.07,0.08,0.07,0.08,0.09,0.07,0.08
Males 15 years and over moe,4207.01,1965.46,1885.63,2002.20,1806.79,2468.05,1961.82,2153.23,2184.99,1512.27,1665.35
Males 15 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
Never married counts,61805.92,24243.33,21607.23,22119.34,20300.95,24438.07,19503.13,23512.77,29915.61,15291.07,19171.56
Never married counts percent,0.09,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.02,0.03
Never married moe,3024.73,1451.43,1433.59,1526.66,1395.27,1682.92,1514.92,1590.69,1812.71,1051.46,1191.13
Never married moa percent,0.03,0.04,0.04,0.04,0.04,0.04,0.05,0.04,0.04,0.04,0.04
"Now married, except separated counts",100721.82,22674.43,19911.80,22826.24,21913.50,19514.69,22892.00,26582.79,24148.47,26508.38,28362.88
"Now married, except separated counts percent",0.14,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.04,0.04


### Fertility

In [9]:
grouping = groups[3]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final4 = cal_district_numbers(dist, dataall, labels, labels_check, 'dp02_0037e' )
final4

FERTILITY

Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 15 to 50 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 15 to 19 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 20 to 34 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 35 to 50 years old


,0,1,2,3,4,5,6,7,8,9,10
Number of women 15 to 50 years old who had a birth in the past 12 months counts,6623.88,1734.00,2280.79,1836.74,1734.03,2278.44,1899.60,2322.96,1903.84,1500.19,1984.52
Number of women 15 to 50 years old who had a birth in the past 12 months counts percent,0.26,0.07,0.09,0.07,0.07,0.09,0.07,0.09,0.07,0.06,0.08
Number of women 15 to 50 years old who had a birth in the past 12 months moe,634.19,357.77,394.20,338.82,325.06,349.99,457.02,404.19,367.38,325.42,377.60
Number of women 15 to 50 years old who had a birth in the past 12 months moa percent,0.06,0.13,0.11,0.11,0.11,0.09,0.15,0.11,0.12,0.13,0.12
"Unmarried women (widowed, divorced, and never married) counts",1899.56,630.24,1097.63,795.62,818.94,1198.45,1038.74,1070.04,657.31,400.10,422.37
"Unmarried women (widowed, divorced, and never married) counts percent",0.07,0.02,0.04,0.03,0.03,0.05,0.04,0.04,0.03,0.02,0.02
"Unmarried women (widowed, divorced, and never married) moe",393.25,247.12,285.14,218.48,256.85,257.87,416.80,306.57,253.99,180.48,180.26
"Unmarried women (widowed, divorced, and never married) moa percent",0.13,0.24,0.16,0.17,0.19,0.13,0.24,0.17,0.23,0.27,0.26
"Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women counts",2554.91,910.89,1398.21,1317.95,869.31,1958.45,989.58,1253.69,638.68,629.12,611.21
"Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women counts percent",0.10,0.04,0.05,0.05,0.03,0.08,0.04,0.05,0.02,0.02,0.02


### Grandparents

In [10]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final5 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0044E')
final5

GRANDPARENTS

Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years
Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years
Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren un

,0,1,2,3,4,5,6,7,8,9,10
Number of grandparents living with own grandchildren under 18 years counts,14092.53,2818.54,4339.91,5933.40,5233.60,6206.07,4570.83,4546.91,1988.72,1727.03,2963.46
Number of grandparents living with own grandchildren under 18 years counts percent,0.26,0.05,0.08,0.11,0.10,0.11,0.08,0.08,0.04,0.03,0.05
Number of grandparents living with own grandchildren under 18 years moe,1221.75,424.27,601.99,775.53,668.26,719.91,710.05,662.36,464.52,481.34,547.50
Number of grandparents living with own grandchildren under 18 years moa percent,0.05,0.09,0.08,0.08,0.08,0.07,0.09,0.09,0.14,0.17,0.11
Grandparents responsible for grandchildren counts,4006.42,936.57,1496.34,1919.43,2083.18,1906.50,1302.71,1405.83,695.81,362.68,1159.53
Grandparents responsible for grandchildren counts percent,0.07,0.02,0.03,0.04,0.04,0.04,0.02,0.03,0.01,0.01,0.02
Grandparents responsible for grandchildren moe,575.59,253.20,323.78,463.52,472.66,357.29,421.80,340.13,286.73,176.66,357.86
Grandparents responsible for grandchildren moa percent,0.09,0.16,0.13,0.15,0.14,0.11,0.20,0.15,0.25,0.30,0.19
Years responsible for grandchildren!!Less than 1 year counts,598.16,119.70,170.86,261.20,296.53,309.70,343.53,435.20,83.35,69.79,122.98
Years responsible for grandchildren!!Less than 1 year counts percent,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00


### School Enrollment

In [11]:
grouping = groups[5]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final6 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0053E')
final6

SCHOOL ENROLLMENT

Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school


,0,1,2,3,4,5,6,7,8,9,10
Population 3 years and over enrolled in school counts,138810.77,32419.97,35355.44,36579.61,39123.44,36260.21,35192.68,39321.53,43669.25,31182.38,34894.72
Population 3 years and over enrolled in school counts percent,0.28,0.06,0.07,0.07,0.08,0.07,0.07,0.08,0.09,0.06,0.07
Population 3 years and over enrolled in school moe,4313.86,1771.32,2063.27,1774.81,1925.03,2100.09,2125.61,2341.53,2097.70,1755.66,1930.40
Population 3 years and over enrolled in school moa percent,0.02,0.03,0.04,0.03,0.03,0.04,0.04,0.04,0.03,0.03,0.03
"Nursery school, preschool counts",7496.45,1910.61,2217.17,2023.35,2019.35,1929.83,1620.28,1777.54,1759.92,1933.18,2290.33
"Nursery school, preschool counts percent",0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Nursery school, preschool moe",883.03,346.79,494.00,344.73,386.57,387.48,367.13,353.89,330.35,465.70,416.28
"Nursery school, preschool moa percent",0.07,0.11,0.14,0.10,0.12,0.12,0.14,0.12,0.11,0.15,0.11
Kindergarten counts,7357.81,2005.96,1699.43,2220.32,2245.07,3031.94,1939.82,1817.53,1816.47,1377.36,1626.28
Kindergarten counts percent,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00


### Educational Attainment

In [12]:
grouping = groups[6]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final7 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0059E')
final7

EDUCATIONAL ATTAINMENT

Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher


,0,1,2,3,4,5,6,7,8,9,10
Population 25 years and over counts,305447.24,93100.82,80840.99,90566.78,79404.06,88998.71,81935.45,102030.78,91081.04,83513.15,94739.97
Population 25 years and over counts percent,0.26,0.08,0.07,0.08,0.07,0.07,0.07,0.09,0.08,0.07,0.08
Population 25 years and over moe,5079.33,2605.78,2558.06,2574.11,2352.54,2904.19,2545.13,2756.43,2522.86,2153.78,2281.53
Population 25 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01
Less than 9th grade counts,11680.96,8508.02,7603.67,10439.19,9334.49,17026.53,5074.52,6102.95,2963.23,1381.04,3325.41
Less than 9th grade counts percent,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.00,0.00
Less than 9th grade moe,1065.70,895.50,748.14,903.08,921.82,1083.93,754.01,680.80,601.78,284.91,627.48
Less than 9th grade moa percent,0.06,0.06,0.06,0.05,0.06,0.04,0.09,0.07,0.12,0.13,0.11
"9th to 12th grade, no diploma counts",16553.75,9603.67,8991.07,13109.82,10743.58,16609.92,4777.23,7111.96,3161.57,2944.35,4084.08
"9th to 12th grade, no diploma counts percent",0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.00,0.00


### Veteran status

In [13]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final8 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0069E')
final8

VETERAN STATUS

Estimate!!VETERAN STATUS!!Civilian population 18 years and over
Estimate!!VETERAN STATUS!!Civilian population 18 years and over!!Civilian veterans


,0,1,2,3,4,5,6,7,8,9,10
Civilian population 18 years and over counts,339035.17,107905.51,92102.37,103774.74,92168.54,102554.91,94478.85,116757.62,116029.84,91648.17,106971.27
Civilian population 18 years and over counts percent,0.25,0.08,0.07,0.08,0.07,0.08,0.07,0.09,0.09,0.07,0.08
Civilian population 18 years and over moe,5786.80,2919.57,2932.62,2965.09,2759.92,3257.03,2906.51,3174.97,2955.73,2467.44,2584.19
Civilian population 18 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01
Civilian veterans counts,45951.70,7139.73,8665.95,7281.63,7266.08,4176.25,9456.77,10524.56,8961.50,9618.89,12704.93
Civilian veterans counts percent,0.03,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01
Civilian veterans moe,1948.19,624.65,691.19,667.54,562.88,396.98,803.05,828.57,760.27,755.49,773.56
Civilian veterans moa percent,0.03,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.05,0.05,0.04


### DISABILITY STATUS

In [14]:
grouping = groups[8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final9 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0071E')
final9

DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION

Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Under 18 years
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Under 18 years!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!18 to 64 years
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!18 to 64 years!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!65 years and over
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!65 years and over!!With a disability


,0,1,2,3,4,5,6,7,8,9,10
Total Civilian Noninstitutionalized Population counts,469215.23,135947.01,127336.16,139627.00,131390.02,137979.32,128793.15,151161.86,143873.70,119221.97,139904.59
Total Civilian Noninstitutionalized Population counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.08
Total Civilian Noninstitutionalized Population moe,8069.34,3897.42,4233.04,4025.34,4157.28,4407.51,4404.78,4548.57,3800.39,3347.23,3797.66
Total Civilian Noninstitutionalized Population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
With a disability counts,57403.73,20058.71,21615.51,27176.99,23874.83,27855.86,16284.85,22167.27,15392.62,12440.25,18202.37
With a disability counts percent,0.03,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
With a disability moe,2402.32,1128.36,1410.46,1617.47,1651.90,1477.84,1351.58,1426.13,1231.62,995.69,1316.61
With a disability moa percent,0.03,0.03,0.04,0.04,0.04,0.03,0.05,0.04,0.05,0.05,0.04
Under 18 years counts,133769.00,29011.79,35821.67,36735.81,39606.58,38311.41,34429.30,35184.62,28392.46,27749.69,33321.68
Under 18 years counts percent,0.07,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02


### RESIDENCE 1 YEAR AGO

In [15]:
grouping = groups[9]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final10 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0079E')
final10

RESIDENCE 1 YEAR AGO

Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Same house
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Same county
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county!!Same state
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county!!Different state
Estimat

,0,1,2,3,4,5,6,7,8,9,10
Population 1 year and over counts,478560.46,136271.32,128762.19,139412.49,131352.05,139645.98,127862.80,150925.26,143478.12,119269.02,139219.31
Population 1 year and over counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.08
Population 1 year and over moe,8574.05,3866.50,4221.23,3995.02,4138.07,4644.30,4221.55,4554.61,3765.28,3295.27,3778.80
Population 1 year and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Same house counts,405809.72,113640.25,107150.50,122597.76,115806.25,122499.60,108612.63,126064.62,101219.39,96816.19,118573.10
Same house counts percent,0.22,0.06,0.06,0.07,0.06,0.07,0.06,0.07,0.06,0.05,0.06
Same house moe,8032.83,3615.27,4075.99,3933.53,4097.01,4191.53,4063.38,4239.16,3369.80,3063.06,3621.49
Same house moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Different house (in the U.S. or abroad) counts,72750.74,22631.07,21611.69,16814.73,15545.80,17146.38,19250.18,24860.63,42258.73,22452.83,20646.21
Different house (in the U.S. or abroad) counts percent,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01


### Place of birth

In [16]:
grouping = groups[10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final11 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0088E')
final11

PLACE OF BIRTH

Estimate!!PLACE OF BIRTH!!Total population
Estimate!!PLACE OF BIRTH!!Total population!!Native
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States!!State of residence
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States!!Different state
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in Puerto Rico, U.S. Island areas, or born abroad to American parent(s)
Estimate!!PLACE OF BIRTH!!Total population!!Foreign born


,0,1,2,3,4,5,6,7,8,9,10
Total population counts,484420.83,137461.90,130625.49,140848.63,132734.78,141064.66,129776.89,152535.62,145193.17,120314.07,141125.95
Total population counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.06,0.08
Total population moe,8608.46,3922.65,4282.35,4040.20,4164.93,4677.30,4401.06,4572.60,3805.53,3340.46,3831.35
Total population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Native counts,434993.19,112535.54,113609.61,123374.74,112439.17,114992.09,115337.70,133168.97,118173.73,105864.10,125679.16
Native counts percent,0.23,0.06,0.06,0.07,0.06,0.06,0.06,0.07,0.06,0.06,0.07
Native moe,8131.85,3498.91,4020.14,3695.61,3811.40,4342.38,4275.35,4174.72,3430.02,3153.59,3668.16
Native moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Native!!Born in United States counts,418640.99,109999.84,109840.21,121861.24,109835.66,113615.63,111507.75,129889.49,114533.59,102062.86,121434.75
Native!!Born in United States counts percent,0.23,0.06,0.06,0.07,0.06,0.06,0.06,0.07,0.06,0.05,0.07


### U.S. CITIZENSHIP STATUS

In [17]:
grouping = groups[11]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final12 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0095E')
final12

U.S. CITIZENSHIP STATUS

Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population
Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Naturalized U.S. citizen
Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen


,0,1,2,3,4,5,6,7,8,9,10
Foreign-born population counts,49427.64,24926.36,17015.88,17473.89,20295.61,26072.58,14439.20,19366.64,27019.44,14449.97,15446.80
Foreign-born population counts percent,0.20,0.10,0.07,0.07,0.08,0.11,0.06,0.08,0.11,0.06,0.06
Foreign-born population moe,2548.74,1885.92,1430.34,1545.07,1464.79,1717.69,1430.51,1746.60,2092.81,1240.44,1466.82
Foreign-born population moa percent,0.03,0.05,0.05,0.05,0.04,0.04,0.06,0.05,0.05,0.05,0.06
Naturalized U.S. citizen counts,27320.53,7632.28,6212.77,6439.72,8683.88,7684.42,7103.60,9688.48,10772.55,7695.33,7886.45
Naturalized U.S. citizen counts percent,0.11,0.03,0.03,0.03,0.04,0.03,0.03,0.04,0.04,0.03,0.03
Naturalized U.S. citizen moe,1783.00,761.42,708.31,808.90,819.38,772.84,857.68,1307.66,1046.32,700.71,810.32
Naturalized U.S. citizen moa percent,0.04,0.06,0.07,0.08,0.06,0.06,0.07,0.08,0.06,0.06,0.06
Not a U.S. citizen counts,22107.11,17294.07,10803.11,11034.17,11611.74,18388.16,7335.59,9678.17,16246.89,6754.64,7560.35
Not a U.S. citizen counts percent,0.09,0.07,0.04,0.04,0.05,0.07,0.03,0.04,0.07,0.03,0.03


### YEAR OF ENTRY

In [18]:
grouping = groups[12]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final13 = cal_district_numbers(dist, dataall, labels, labels_check,'DP02_0098E')
final13

YEAR OF ENTRY

Estimate!!YEAR OF ENTRY!!Population born outside the United States
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native!!Entered 2010 or later
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native!!Entered before 2010
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born!!Entered 2010 or later
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born!!Entered before 2010


,0,1,2,3,4,5,6,7,8,9,10
Population born outside the United States counts,65779.84,27462.06,20785.28,18987.40,22899.12,27449.04,18269.14,22646.12,30659.58,18251.21,19691.21
Population born outside the United States counts percent,0.22,0.09,0.07,0.06,0.08,0.09,0.06,0.08,0.10,0.06,0.07
Population born outside the United States moe,2901.47,1953.06,1540.49,1585.30,1535.44,1779.11,1580.02,1850.21,2147.79,1348.55,1639.79
Population born outside the United States moa percent,0.03,0.04,0.05,0.05,0.04,0.04,0.05,0.05,0.04,0.04,0.05
Native counts,16352.20,2535.70,3769.40,1513.51,2603.50,1376.46,3829.95,3279.48,3640.14,3801.25,4244.41
Native counts percent,0.06,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01
Native moe,1310.18,532.73,564.87,345.07,459.03,362.80,683.22,479.83,505.53,600.74,695.52
Native moa percent,0.05,0.13,0.09,0.14,0.11,0.16,0.11,0.09,0.08,0.10,0.10
Native!!Entered 2010 or later counts,4874.55,462.18,1437.58,513.41,597.05,488.26,1033.93,938.28,504.63,933.99,857.15
Native!!Entered 2010 or later counts percent,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### WORLD REGION OF BIRTH OF FOREIGN BORN

In [19]:
grouping = groups[13]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final14 = cal_district_numbers(dist, dataall, labels, labels_check,'DP02_0105E')
final14

WORLD REGION OF BIRTH OF FOREIGN BORN

Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Europe
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Asia
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Africa
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Oceania
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Latin America
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America


/Users/mists/Projects/sa2020/sa2020-ccd/ccd_functions.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)


,0,1,2,3,4,5,6,7,8,9,10
"Foreign-born population, excluding population born at sea counts",49427.64,24926.36,17015.88,17473.89,20295.61,26072.58,14439.20,19366.64,27019.44,14449.97,15446.80
"Foreign-born population, excluding population born at sea counts percent",0.20,0.10,0.07,0.07,0.08,0.11,0.06,0.08,0.11,0.06,0.06
"Foreign-born population, excluding population born at sea moe",2548.74,1885.92,1430.34,1545.07,1464.79,1717.69,1430.51,1746.60,2092.81,1240.44,1466.82
"Foreign-born population, excluding population born at sea moa percent",0.03,0.05,0.05,0.05,0.04,0.04,0.06,0.05,0.05,0.05,0.06
Europe counts,3238.46,680.36,563.20,123.59,323.52,200.36,430.24,995.49,1414.40,1206.76,1186.62
Europe counts percent,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
Europe moe,544.50,182.71,193.42,97.96,142.16,147.97,159.35,242.78,319.22,263.37,256.31
Europe moa percent,0.10,0.16,0.21,0.48,0.27,0.45,0.23,0.15,0.14,0.13,0.13
Asia counts,10385.49,1701.06,1568.63,530.33,1142.93,224.89,2141.02,5501.49,13241.20,4900.08,3469.88
Asia counts percent,0.04,0.01,0.01,0.00,0.00,0.00,0.01,0.02,0.05,0.02,0.01


### LANGUAGE SPOKEN AT HOME

In [20]:
grouping = groups[14]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final15 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP02_0112E')
final15

LANGUAGE SPOKEN AT HOME

Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!English only
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Language other than English
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Language other than English!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Spanish
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Spanish!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other Indo-European languages
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other Indo-European languages!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Asian and Pacific Islander languages
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Asian and Pacific Islander languages!!Speak

,0,1,2,3,4,5,6,7,8,9,10
Population 5 years and over counts,450001.97,129282.86,120286.23,131649.72,123005.99,130548.27,120342.26,142756.64,136289.39,114200.09,131719.56
Population 5 years and over counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.08
Population 5 years and over moe,8114.10,3673.57,3840.05,3753.91,3781.45,4209.08,3910.51,4226.35,3562.71,3086.36,3498.16
Population 5 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
English only counts,316191.02,69191.29,73458.94,68477.55,59655.55,46769.42,72081.13,87635.63,83015.03,84091.52,96677.90
English only counts percent,0.18,0.04,0.04,0.04,0.03,0.03,0.04,0.05,0.05,0.05,0.06
English only moe,7076.95,2810.61,2990.62,2855.04,2966.51,2792.75,3135.68,3602.92,2900.60,2733.61,3172.43
English only moa percent,0.01,0.02,0.02,0.03,0.03,0.04,0.03,0.02,0.02,0.02,0.02
Language other than English counts,133810.95,60091.58,46827.29,63172.16,63350.44,83778.85,48261.13,55121.01,53274.36,30108.58,35041.65
Language other than English counts percent,0.08,0.03,0.03,0.04,0.04,0.05,0.03,0.03,0.03,0.02,0.02


### ANCESTRY

In [21]:
grouping = groups[15]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final16 = cal_district_numbers(dist, dataall, labels, labels_check,'DP02_0124E')
final16

ANCESTRY

Estimate!!ANCESTRY!!Total population
Estimate!!ANCESTRY!!Total population!!American
Estimate!!ANCESTRY!!Total population!!Arab
Estimate!!ANCESTRY!!Total population!!Czech
Estimate!!ANCESTRY!!Total population!!Danish
Estimate!!ANCESTRY!!Total population!!Dutch
Estimate!!ANCESTRY!!Total population!!English
Estimate!!ANCESTRY!!Total population!!French (except Basque)
Estimate!!ANCESTRY!!Total population!!French Canadian
Estimate!!ANCESTRY!!Total population!!German
Estimate!!ANCESTRY!!Total population!!Greek
Estimate!!ANCESTRY!!Total population!!Hungarian
Estimate!!ANCESTRY!!Total population!!Irish
Estimate!!ANCESTRY!!Total population!!Italian
Estimate!!ANCESTRY!!Total population!!Lithuanian
Estimate!!ANCESTRY!!Total population!!Norwegian
Estimate!!ANCESTRY!!Total population!!Polish
Estimate!!ANCESTRY!!Total population!!Portuguese
Estimate!!ANCESTRY!!Total population!!Russian
Estimate!!ANCESTRY!!Total population!!Scotch-Irish
Estimate!!ANCESTRY!!Total population!!Scottish
Estimat

,0,1,2,3,4,5,6,7,8,9,10
Total population counts,484420.83,137461.90,130625.49,140848.63,132734.78,141064.66,129776.89,152535.62,145193.17,120314.07,141125.95
Total population counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.06,0.08
Total population moe,8608.46,3922.65,4282.35,4040.20,4164.93,4677.30,4401.06,4572.60,3805.53,3340.46,3831.35
Total population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
American counts,16643.48,3228.00,3716.33,4375.38,4001.82,3014.88,4591.37,4691.73,4488.48,6027.23,6690.30
...,...,...,...,...,...,...,...,...,...,...,...
Welsh moa percent,0.15,0.26,0.32,0.86,0.42,3.62,0.32,0.22,0.28,0.38,0.22
West Indian (excluding Hispanic origin groups) counts,1918.74,322.45,489.83,68.17,245.20,59.03,469.15,231.69,394.95,450.53,508.26
West Indian (excluding Hispanic origin groups) counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
West Indian (excluding Hispanic origin groups) moe,684.26,273.77,220.23,102.75,207.77,102.08,290.10,202.41,213.62,263.40,265.35


### COMPUTERS AND INTERNET USE

In [22]:
grouping = groups[16]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final17 = cal_district_numbers(dist, dataall, labels, labels_check,'DP02_0152E')
final17

COMPUTERS AND INTERNET USE

Estimate!!COMPUTERS AND INTERNET USE!!Total households
Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer
Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription


,0,1,2,3,4,5,6,7,8,9,10
Total households counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Total households counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Total households moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
With a computer counts,154698.66,50765.46,42041.64,43058.14,38199.60,38513.35,42807.18,54367.18,59793.41,49533.00,53668.38
With a computer counts percent,0.23,0.08,0.06,0.06,0.06,0.06,0.06,0.08,0.09,0.07,0.08
With a computer moe,2377.35,1451.15,1194.70,1210.18,995.79,1245.69,1249.26,1449.61,1484.72,1209.96,1161.32
With a computer moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01
With a broadband Internet subscription counts,146009.35,44339.37,36486.94,36770.76,35293.08,33704.50,39765.14,50331.73,55684.89,47357.70,48781.53
With a broadband Internet subscription counts percent,0.22,0.07,0.05,0.06,0.05,0.05,0.06,0.08,0.08,0.07,0.07


In [23]:
with pd.ExcelWriter('sa2020_ccd_household_update.xlsx') as writer:
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name=groups[7][:30])
    final9.to_excel(writer, sheet_name=groups[8][:30])
    final10.to_excel(writer, sheet_name=groups[9][:30])
    final11.to_excel(writer, sheet_name=groups[10][:30])        
    final12.to_excel(writer, sheet_name=groups[11][:30])
    final13.to_excel(writer, sheet_name=groups[12][:30])
    final14.to_excel(writer, sheet_name=groups[13][:30])
    final15.to_excel(writer, sheet_name=groups[14][:30])
    final16.to_excel(writer, sheet_name=groups[15][:30])
    final17.to_excel(writer, sheet_name=groups[16][:30])